In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class StockDataFetcher:
    """Handles all stock data fetching logic"""
    def __init__(self, exchange_suffix='.NS', buffer_days=30):
        self.exchange_suffix = exchange_suffix
        self.buffer_days = buffer_days

    def fetch(self, symbol, period_days):
        """Returns historical price DataFrame for the given symbol"""
        ticker_symbol = f"{symbol}{self.exchange_suffix}"
        end_date = datetime.now()
        start_date = end_date - timedelta(days=period_days + self.buffer_days)
        try:
            ticker = yf.Ticker(ticker_symbol)
            hist = ticker.history(start=start_date, end=end_date)
            if hist.empty or len(hist) < 10:
                return None
            return hist.tail(period_days) if len(hist) > period_days else hist
        except Exception as e:
            print(f"Error fetching {symbol}: {str(e)}")
            return None

class VolatilityCalculator:
    """Calculates volatility statistics for a price series"""
    @staticmethod
    def annualized_volatility(price_series):
        returns = price_series.pct_change().dropna()
        return returns.std() * np.sqrt(252) * 100  # Percentage

class DrawdownCalculator:
    """Calculates drawdown for a price series"""
    @staticmethod
    def max_drawdown(price_series):
        cumulative = (1 + price_series.pct_change()).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        return drawdown.min() * 100  # Percentage

class PortfolioAnalyzer:
    """Analyzes a portfolio DataFrame and runs analytics functions"""
    def __init__(self, stock_fetcher):
        self.stock_fetcher = stock_fetcher

    def analyze_individual_stock(self, symbol, period_days):
        hist = self.stock_fetcher.fetch(symbol, period_days)
        if hist is None or hist.empty:
            return None, None
        vol = VolatilityCalculator.annualized_volatility(hist['Close'])
        dd = DrawdownCalculator.max_drawdown(hist['Close'])
        return vol, dd

    def analyze_portfolio(self, portfolio_df, period_days, quantity_col='CurrentQuantity'):
        stock_weights = {}
        total_value = 0
        valid_stocks = []
        stock_returns = {}
        for idx, row in portfolio_df.iterrows():
            symbol = row['Symbol']
            quantity = row[quantity_col]
            hist = self.stock_fetcher.fetch(symbol, period_days)
            if hist is not None and not hist.empty:
                current_price = hist['Close'].iloc[-1]
                value = current_price * quantity
                total_value += value
                stock_returns[symbol] = hist['Close'].pct_change().dropna()
                valid_stocks.append({
                    'Symbol': symbol,
                    'Quantity': quantity,
                    'CurrentPrice': current_price,
                    'Value': value
                })
        if total_value == 0 or not valid_stocks:
            return None, None, None, 0
        for stock in valid_stocks:
            stock['Weight'] = stock['Value'] / total_value
            stock_weights[stock['Symbol']] = stock['Weight']
        returns_df = pd.DataFrame(stock_returns).dropna()
        if returns_df.empty:
            return None, None, None, total_value
        weights_series = pd.Series(stock_weights)
        portfolio_returns = (returns_df * weights_series).sum(axis=1)
        vol = portfolio_returns.std() * np.sqrt(252) * 100
        cumulative = (1 + portfolio_returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min() * 100
        return vol, max_drawdown, valid_stocks, total_value

def run_analysis(symbols, periods, quantity=1):
    fetcher = StockDataFetcher('.NS', 30)
    analyzer = PortfolioAnalyzer(fetcher)
    # Pre-initialize metrics lists per period
    wv, wdd = {p: [] for p in periods}, {p: [] for p in periods}
    for symbol in symbols:
        portfolio_df = pd.DataFrame({"Symbol": [symbol], "CurrentQuantity": [quantity]})
        for period_name, days in periods.items():
            # Individual Stock
            vol, dd = analyzer.analyze_individual_stock(symbol, days)
            if vol is not None and dd is not None:
                print(f"{symbol} {period_name}: Volatility={vol:.2f}%, MaxDrawdown={dd:.2f}%")
            else:
                print(f"{symbol} {period_name}: No data")
            # Portfolio-level metrics (trivial for single stock, extendable to multi-stock batch)
            p_vol, p_dd, stock_info, total_val = analyzer.analyze_portfolio(portfolio_df, days)
            if p_vol is not None and p_dd is not None:
                print(f"Portfolio {symbol} {period_name}: Weighted Volatility={p_vol:.2f}%, Weighted MaxDrawdown={p_dd:.2f}%")
                wv[period_name].append(p_vol)
                wdd[period_name].append(p_dd)
    return wv, wdd

if __name__ == "__main__":
    sec = [    "NIFTYBEES",
    "BANKBEES",
    "MID150BEES",
    "HDFCSML250"]
    periods = {'3_months': 90, '6_months': 180, '1_year': 365}
    wv, wdd = run_analysis(sec, periods, quantity=1)


NIFTYBEES 3_months: Volatility=7.79%, MaxDrawdown=-3.20%
Portfolio NIFTYBEES 3_months: Weighted Volatility=7.79%, Weighted MaxDrawdown=-3.20%
NIFTYBEES 6_months: Volatility=9.90%, MaxDrawdown=-4.45%
Portfolio NIFTYBEES 6_months: Weighted Volatility=9.90%, Weighted MaxDrawdown=-4.45%
NIFTYBEES 1_year: Volatility=11.75%, MaxDrawdown=-11.13%
Portfolio NIFTYBEES 1_year: Weighted Volatility=11.75%, Weighted MaxDrawdown=-11.13%
BANKBEES 3_months: Volatility=8.54%, MaxDrawdown=-5.70%
Portfolio BANKBEES 3_months: Weighted Volatility=8.54%, Weighted MaxDrawdown=-5.70%
BANKBEES 6_months: Volatility=10.27%, MaxDrawdown=-6.16%
Portfolio BANKBEES 6_months: Weighted Volatility=10.27%, Weighted MaxDrawdown=-6.16%
BANKBEES 1_year: Volatility=13.16%, MaxDrawdown=-10.65%
Portfolio BANKBEES 1_year: Weighted Volatility=13.16%, Weighted MaxDrawdown=-10.65%
MID150BEES 3_months: Volatility=11.55%, MaxDrawdown=-5.53%
Portfolio MID150BEES 3_months: Weighted Volatility=11.55%, Weighted MaxDrawdown=-5.53%
MID150